In [1]:
import numpy as np
import pandas as pd

Import data from UCI Repository 

In [12]:
uris = [
    'http://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data',
    'https://archive.ics.uci.edu/ml/machine-learning-databases/letter-recognition/letter-recognition.data'
]

df_adult, df_letter = [pd.read_csv(uri, header=None) for uri in uris]

df_covtype = pd.read_csv('covtype.data', header=None)

Inspect the ADULT dataset

In [13]:
df_adult.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [9]:
df_adult.columns = ['age, workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'captial-loss', 'hours-per-week', 'native-country','income']

ValueError: Length mismatch: Expected axis has 15 elements, new values have 14 elements